In [ ]:
#IEMD
def earth_mover_distance(y_true,y_pre):
    return tf.reduce_mean(tf.square(tf.square(tf.cumsum(y_true,axis=-1)-tf.cumsum(y_pre,axis=-1))),axis=-1)

In [ ]:
y_pred=[]
y_true=[]
X_test_all=[]

In [ ]:
#SCSAE-IEMD
for train,test in kfold.split(f,y):
    X_train,X_valid=train_test_split(f[train],test_size=DATA_SPLIT_PCT,random_state=SEED)
    X_test=f[test]
    y_train,y_valid=train_test_split(y_cate[train],test_size=DATA_SPLIT_PCT,random_state=SEED)
    y_test=y_cate[test]
    X_test_all.append(X_test)
   
   #Encoder
    input_img=Input(shape=(72, 72, 1))

    Conv2D_1_encoder=Conv2D(filters = 6, kernel_size = (5, 5), activation='relu', padding='same')

    Conv2D_1_decoder=Conv2D(filters=1,kernel_size= (5,5), activation='sigmoid', padding='same')

    Conv2D_2_encoder=Conv2D(filters = 16, kernel_size = (5, 5), activation='relu', padding='same')

    Conv2D_2_decoder=Conv2D(filters=6, kernel_size=(5, 5), activation='relu', padding='same')

    Conv2D_3_encoder=Conv2D(filters = 32, kernel_size = (5, 5), activation='relu', padding='same')

    Conv2D_3_decoder=Conv2D(filters = 16, kernel_size = (5, 5), activation='relu', padding='same')
#
    encoder_1= Conv2D_1_encoder(input_img)

    encoder_1= MaxPooling2D(pool_size = (2, 2), padding='same')(encoder_1)

    #Decoder
    decoder_1= Conv2D(filters=6, kernel_size=(5, 5), activation='relu', padding='same')(encoder_1)

    decoder_1= UpSampling2D( (2, 2))(decoder_1)

    aux_output_1=Conv2D_1_decoder(decoder_1)
    
    flatten_1=Flatten()(encoder_1)

    dense_1=Dense(512,activation="relu")(flatten_1)

    LR_1=Dense(13,activation="softmax",name='LR')(dense_1)

    autoencoder_1=Model(inputs=input_img,outputs=[aux_output_1,LR_1,LR_1])

    #Encoder
    encoder_2=Conv2D_1_encoder(input_img)

    encoder_2= MaxPooling2D(pool_size = (2, 2),padding='same')(encoder_2)

    encoder_2=Conv2D_2_encoder(encoder_2)

    encoder_2= MaxPooling2D(pool_size = (2, 2),padding='same')(encoder_2)

    #Decoder
    decoder_2= Conv2D(16, (5, 5), activation='relu', padding='same')(encoder_2)

    decoder_2= UpSampling2D( (2, 2))(decoder_2)

    decoder_2=  Conv2D_2_decoder(decoder_2)

    decoder_2= UpSampling2D( (2, 2))(decoder_2)

    aux_output_2=Conv2D_1_decoder(decoder_2)

    flatten_2=Flatten()(encoder_2)

    dense_2=Dense(512,activation="relu")(flatten_2)

    LR_2=Dense(13,activation="softmax",name='LR')(dense_2)

    autoencoder_2=Model(inputs=input_img,outputs=[aux_output_2,LR_2,LR_2])

    #Encoder
    encoder_3=Conv2D_1_encoder(input_img)

    encoder_3= MaxPooling2D(pool_size = (2, 2),padding='same')(encoder_3)

    encoder_3=Conv2D_2_encoder(encoder_3)

    encoder_3= MaxPooling2D(pool_size = (2, 2),padding='same')(encoder_3)

    encoder_3=Conv2D_3_encoder(encoder_3)

    encoder_3= MaxPooling2D(pool_size = (2, 2),padding='same')(encoder_3)

    #Decoder
    decoder_3= Conv2D(32, (5, 5), activation='relu', padding='same')(encoder_3)

    decoder_3= UpSampling2D( (2, 2))(decoder_3)

    decoder_3= Conv2D_3_decoder(decoder_3)

    decoder_3= UpSampling2D( (2, 2))(decoder_3)

    decoder_3=  Conv2D_2_decoder(decoder_3)

    decoder_3= UpSampling2D( (2, 2))(decoder_3)

    aux_output_3=Conv2D_1_decoder(decoder_3)

    flatten_3=Flatten()(encoder_3)

    dense_3=Dense(256,activation="relu")(flatten_3)
    
    dense_3=Dense(64,activation="relu")(dense_3)
    
    LR_3=Dense(13,activation="softmax",name='LR')(dense_3)

    autoencoder_3=Model(inputs=input_img,outputs=[aux_output_3,LR_3,LR_3])
    
    autoencoder_1.compile(metrics=['accuracy'],
                   optimizer=Adam(lr=learning_rate),
                    loss_weights= [0.05,1, 0])

    history  =autoencoder_1.fit(X_train,
                          [X_train, y_train,y_train],
                          validation_data=(X_valid,[X_valid,y_valid,y_valid]),
                          epochs=nb_epoch,
                          batch_size=batch_size,
                          shuffle=True,
                          verbose=0).history

    autoencoder_2.compile(metrics=['accuracy'],
                    loss=['mean_squared_error', earth_mover_distance,'categorical_crossentropy'],
                    optimizer=Adam(lr=learning_rate),
                    loss_weights= [0.05,1, 0])

    history  =autoencoder_2.fit(X_train,
                          [X_train, y_train,y_train],
                          validation_data=(X_valid,[X_valid,y_valid,y_valid]),
                          epochs=nb_epoch,
                          batch_size=batch_size,
                          shuffle=True,
                          verbose=0).history

    autoencoder_3.compile(metrics=['accuracy'],
                    loss=['mean_squared_error', earth_mover_distance,'categorical_crossentropy'],
                    optimizer=Adam(lr=learning_rate),
                    loss_weights= [0.05,1, 0])
    cp = ModelCheckpoint(filepath="EEMD-SHAPEALL-loss.hdf5",monitor='val_LR_1_loss',save_best_only=True,
                     verbose = 1,mode='min')
    callbacks_list=[cp]

    history = autoencoder_3.fit(X_train,
                          [X_train, y_train,y_train],
                          validation_data=(X_valid,[X_valid,y_valid,y_valid]),
                          epochs=nb_epoch,
                          batch_size=batch_size,
                          shuffle=True,
                          verbose=1,
                          callbacks=callbacks_list).history
    
    autoencoder_3.load_weights("EEMD-SHAPEALL-loss.hdf5")


#Acc
    scores=autoencoder_3.evaluate(X_test,[X_test,y_test,y_test],verbose=1)
    y_pred_1=autoencoder_3.predict_generator(X_test)
    y_pred_class=np.argmax(y_pred_1[1],axis=1)
    y_pred.append(y_pred_class)
    y_true_class=np.argmax(y_test,axis=1)
    y_true.append(y_true_class)
    print(scores[5]*100)
    
    score.append(scores[5]*100)

print( np.mean(score),np.std(score))

In [ ]:
#Confusion Matrix
from numpy import *
import matplotlib.pyplot as plt
from pylab import *
import matplotlib.ticker as ticker

def CM(data, name):
    
    conf_mx = data


    class Percent(float):
        def __str__(self):
            return '{:.2%}'.format(self)

    norm_conf = []
    for i in conf_mx:
        a = 0
        tmp_arr = []
        a = sum(i,0)
        for j in i:
            tmp_arr.append(float(j)/float(a))
        norm_conf.append(tmp_arr)


    plt.clf()
    plt.rcParams['xtick.bottom'] = plt.rcParams['xtick.labelbottom'] = False
    plt.rcParams['xtick.top'] = plt.rcParams['xtick.labeltop'] = True
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['font.serif'] = ['Times New Roman'] + plt.rcParams['font.serif']
    plt.rc('xtick', labelsize='25')
    plt.rc('ytick', labelsize='25')

    rc('font', weight='bold')

    fig = plt.figure(figsize=(18,18))

    COLOR = 'black'
    mpl.rcParams['text.color'] = COLOR  
    ax = fig.add_subplot(111)


    ticks_x = ticker.FuncFormatter(lambda x, pos: 'h'+'{0:g}'.format(x+1))
    ax.xaxis.set_major_formatter(ticks_x)

    ticks_y = ticker.FuncFormatter(lambda x, pos: 'h'+'{0:g}'.format(x+1))
    ax.yaxis.set_major_formatter(ticks_y)

    ax.set_xticks(np.arange(13))
    ax.set_yticks(np.arange(13))

    res = ax.imshow(array(norm_conf), cmap=cm.binary, interpolation='nearest')
    COLOR = 'white'
    mpl.rcParams['text.color'] = COLOR
    for i, cas in enumerate(conf_mx):
        for j, c in enumerate(cas):
            if (c!=0) & (i==j):


                plt.text(j-.2, i-.08, c, fontsize=14, fontweight="extra bold")
                plt.text(j-.38, i+.23, Percent(norm_conf[i][j]), fontsize=13, fontweight="extra bold")


    COLOR = 'black'
    mpl.rcParams['text.color'] = COLOR             
    for i, cas in enumerate(conf_mx):
        for j, c in enumerate(cas):

            if (c!=0) & (i!=j):

                plt.text(j-.2, i-.08, c, fontsize=14, fontweight="extra bold")
                plt.text(j-.38, i+.23, Percent(norm_conf[i][j]), fontsize=13, fontweight="extra bold")  

    base_path = "/home/haptic/ycx/ycx/CM/"
    plt.savefig(base_path + name + ".pdf", format='pdf',edgecolor='none', dpi=300)
    plt.show()

In [ ]:
#Plot the Confusion Matrix
y_true=np.array(y_true).reshape(2340)
y_pred=np.array(y_pred).reshape(2340)
con_mat=confusion_matrix(y_true,y_pred)
CM(con_mat,"EEMD-SHAPEALL")

In [ ]:
#QWK
qwk=cohen_kappa_score(y_true,y_pred,weights='quadratic')
print(qwk)